# Linge & Langtagen, "Programming for Computations"
## Ch. 3.7 Double and triple integrals

### 3.7.3 Monte Carlo integration for complex-shaped domains

**The Monte Carlo integration algorithm**

The idea of Monte Carlo integration of $\int_a^b f(x)dx$ is to use the mean-value theorem from calculus, which states that the integral $\int_a^b f(x)dx$ equals the length of the integration domain, here $b-a$, times the *average* value of $f$, $\bar f$, in $[a,b]$.
The average value can be computed by sampling $f$ at a set of *random* points inside the domain and take the mean of the function values. In higher dimensions, an integral is estimated as the area/volume of the domain times the average value, and again one can evaluate the integrand at a set of random points in the domain and compute the mean value of those evaluations.

Let us introduce some quantities to help us make the specification of the integration algorithm more precise. Suppose we have some two-dimensional integral

$$\int_\Omega f(x,y)dxdx,$$


where $\Omega$  is a two-dimensional domain defined via a help function $g(x,y)$:

$$\Omega = \{ (x,y)\,|\, g(x,y) \geq 0\}$$

That is, points $(x,y)$ for which $g(x,y)≥0$ lie inside $\Omega$, and points for which $g(x,y)<\Omega$ are outside $\Omega$. The boundary of the domain $\partial\Omega$ is given by the implicit curve $g(x,y)=0$. Such formulations of geometries have been very common during the last couple of decades, and one refers to $g$ as a level-set function and the boundary $g=0$ as the zero-level contour of the level-set function. For simple geometries one can easily construct $g$ by hand, while in more complicated industrial applications one must resort to mathematical models for constructing $g$.

Let $A(\Omega)$ be the area of a domain $\Omega$. We can estimate the integral by this Monte Carlo integration method:

1. embed the geometry $\Omega$ in a rectangular area $R$
2. draw a large number of *random* points $(x,y)$ in $R$
3. count the fraction $q$ of points that are inside $\Omega$
4. approximate $A(\Omega)A(R)$ by $q$, i.e., set $A(\Omega)=qA(R)$
5. evaluate the mean of $f$, $\bar f$, at the points inside $\Omega$
6. estimate the integral as $A(\Omega)\bar f$ 

Note that $A(R)$ is trivial to compute since $R$ is a rectangle, while $A(\Omega)$ is unknown. However, if we assume that the fraction of $A(R)$ occupied by $A(\Omega)$ is the same as the fraction of random points inside $\Omega$, we get a simple estimate for $A(\Omega)$.
To get an idea of the method, consider a circular domain $\Omega$ embedded in a rectangle as shown below. A collection of random points is illustrated by black dots.

**Implementation**

A Python function implementing $\int_\Omega f(x,y)dxdy$ can be written like this:

**Verification**

A simple test case is to check the area of a rectangle $[0,2]\times[3,4.5]$ embedded in a rectangle $[0,3]\times[2,5]$. The right answer is 3, but Monte Carlo integration is, unfortunately, never exact so it is impossible to predict the output of the algorithm. All we know is that the estimated integral should approach 3 as the number of random points goes to infinity. Also, for a fixed number of points, we can run the algorithm several times and get different numbers that fluctuate around the exact value, since different sample points are used in different calls to the Monte Carlo integration algorithm.

he area of the rectangle can be computed by the integral $\int_0^2\int_3^{4.5}
dydx$, so in this case we identify $f(x,y)=1$, and the g function can be specified as (e.g.) $1$ if $(x,y)$ is inside $[0,2]\times[3,4.5]$ and $−1$ otherwise. Here is an example on how we can utilize the MonteCarlo_double function to compute the area for different number of samples:

In [2]:
import numpy as np

def MonteCarlo_double(f, g, x0, x1, y0, y1, n):
    """
    Monte Carlo integration of f over a domain g>=0, embedded
    in a rectangle [x0,x1]x[y0,y1]. n^2 is the number of
    random points.
    """
    # Draw n**2 random points in the rectangle
    x = np.random.uniform(x0, x1, n)
    y = np.random.uniform(y0, y1, n)
    # Compute sum of f values inside the integration domain
    f_mean = 0
    num_inside = 0   # number of x,y points inside domain (g>=0)
    for i in range(len(x)):
        for j in range(len(y)):
            if g(x[i], y[j]) >= 0:
                num_inside += 1
                f_mean += f(x[i], y[j])
    f_mean = f_mean/float(num_inside)
    area = num_inside/float(n**2)*(x1 - x0)*(y1 - y0)
    return area*f_mean

def g(x, y):
    return (1 if (0 <= x <= 2 and 3 <= y <= 4.5) else -1)

p1 = MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 100)
p2 = MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 1000)
p3 = MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 2000)
p4 = MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 5000)
 
print('MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 100)')
print('%.16f' %p1)
print('MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 1000)')
print('%.16f' %p2)
print('MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 2000)')
print('%.16f' %p3)
print('MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 5000)')
print('%.16f' %p4)

MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 100)
2.3606999999999996
MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 1000)
3.1389120000000004
MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 2000)
2.9138399999999995
MonteCarlo_double(lambda x, y: 1, g, 0, 3, 2, 5, 5000)
2.9822612400000006


In [ ]:
**Test function for function with random numbers**

To make a test function, we need a unit test that has identical behavior each time we run the test. This seems difficult when random numbers are involved, because these numbers are different every time we run the algorithm, and each run hence produces a (slightly) different result. A standard way to test algorithms involving random numbers is to fix the seed of the random number generator. Then the sequence of numbers is the same every time we run the algorithm. Assuming that the MonteCarlo_double function works, we fix the seed, observe a certain result, and take this result as the correct result. Provided the test function always uses this seed, we should get exactly this result every time the MonteCarlo_double function is called. Our test function can then be written as shown below.

def test_MonteCarlo_double_rectangle_area():
    """Check the area of a rectangle."""
    def g(x, y):
        return (1 if (0 <= x <= 2 and 3 <= y <= 4.5) else -1)

    x0 = 0;  x1 = 3;  y0 = 2;  y1 = 5  # embedded rectangle
    n = 1000
    np.random.seed(8)      # must fix the seed!
    I_expected = 3.121092  # computed with this seed
    I_computed = MonteCarlo_double(
        lambda x, y: 1, g, x0, x1, y0, y1, n)
    assert abs(I_expected - I_computed) < 1E-14

**Integral over a circle**

he test above involves a trivial function $f(x,y)=1$. We should also test a non-constant $f$ function and a more complicated domain. Let $\Omega$ be a circle at the origin with radius 2, and let $f=\sqrt{x^2+y^2}$. This choice makes it possible to compute an exact result: in polar coordinates, $\int_\Omega f(x,y)dxdy$ simplifies to $2\pi\int_0^2 r^2dr = 16\pi/3$. We must be prepared for quite crude approximations that fluctuate around this exact result. As in the test case above, we experience better results with larger number of points. When we have such evidence for a working implementation, we can turn the test into a proper test function. Here is an example:

def test_MonteCarlo_double_circle_r():
    """Check the integral of r over a circle with radius 2."""
    def g(x, y):
        xc, yc = 0, 0  # center
        R = 2          # radius
        return  R**2 - ((x-xc)**2 + (y-yc)**2)

    # Exact: integral of r*r*dr over circle with radius R becomes
    # 2*pi*1/3*R**3
    import sympy
    r = sympy.symbols('r')
    I_exact = sympy.integrate(2*sympy.pi*r*r, (r, 0, 2))
    print ('Exact integral:', I_exact.evalf())
    x0 = -2;  x1 = 2;  y0 = -2;  y1 = 2
    n = 1000
    np.random.seed(6)
    I_expected = 16.7970837117376384  # Computed with this seed
    I_computed = MonteCarlo_double(
        lambda x, y: np.sqrt(x**2 + y**2),
        g, x0, x1, y0, y1, n)
    print ('MC approximation %d samples): %.16f' % (n**2, I_computed))
    assert abs(I_expected - I_computed) < 1E-15

In [ ]:
import numpy as np

def MonteCarlo_double(f, g, x0, x1, y0, y1, n):
    """
    Monte Carlo integration of f over a domain g>=0, embedded
    in a rectangle [x0,x1]x[y0,y1]. n^2 is the number of
    random points.
    """
    # Draw n**2 random points in the rectangle
    x = np.random.uniform(x0, x1, n)
    y = np.random.uniform(y0, y1, n)
    # Compute sum of f values inside the integration domain
    f_mean = 0
    num_inside = 0   # number of x,y points inside domain (g>=0)
    for i in range(len(x)):
        for j in range(len(y)):
            if g(x[i], y[j]) >= 0:
                num_inside += 1
                f_mean += f(x[i], y[j])
    f_mean = f_mean/float(num_inside)
    area = num_inside/float(n**2)*(x1 - x0)*(y1 - y0)
    return area*f_mean

def test_MonteCarlo_double_rectangle_area():
    """Check the area of a rectangle."""
    def g(x, y):
        return (1 if (0 <= x <= 2 and 3 <= y <= 4.5) else -1)

    x0 = 0;  x1 = 3;  y0 = 2;  y1 = 5  # embedded rectangle
    n = 1000
    np.random.seed(8)      # must fix the seed!
    I_expected = 3.121092  # computed with this seed
    I_computed = MonteCarlo_double(
        lambda x, y: 1, g, x0, x1, y0, y1, n)
    assert abs(I_expected - I_computed) < 1E-14

def test_MonteCarlo_double_circle_r():
    """Check the integral of r over a circle with radius 2."""
    def g(x, y):
        xc, yc = 0, 0  # center
        R = 2          # radius
        return  R**2 - ((x-xc)**2 + (y-yc)**2)

    # Exact: integral of r*r*dr over circle with radius R becomes
    # 2*pi*1/3*R**3
    import sympy
    r = sympy.symbols('r')
    I_exact = sympy.integrate(2*sympy.pi*r*r, (r, 0, 2))
    print ('Exact integral:', I_exact.evalf())
    x0 = -2;  x1 = 2;  y0 = -2;  y1 = 2
    n = 1000
    np.random.seed(6)
    I_expected = 16.7970837117376384  # Computed with this seed
    I_computed = MonteCarlo_double(
        lambda x, y: np.sqrt(x**2 + y**2),
        g, x0, x1, y0, y1, n)
    print ('MC approximation %d samples: %.16f' % (n**2, I_computed))
    assert abs(I_expected - I_computed) < 1E-15

if __name__ == '__main__':
    test_MonteCarlo_double_rectangle_area()
    test_MonteCarlo_double_circle_r()